## Purpose
This is a gradual approach on using torchtext. 

#### Tokenize and numericalize
We first define a `Field` which is datatype to process text

In [18]:
import torchtext

field = torchtext.data.Field()

txt = 'This is simple sentence .'

#tokenize 
tokens = field.tokenize(txt)
print(f'tokens: {tokens}')

# build the vocabulary
field.build_vocab(tokens)

tokens: ['This', 'is', 'simple', 'sentence', '.']


Lets have a look at the vocab frequency. 

In [19]:
field.vocab.freqs

Counter({'T': 1,
         'h': 1,
         'i': 3,
         's': 4,
         'm': 1,
         'p': 1,
         'l': 1,
         'e': 4,
         'n': 2,
         't': 1,
         'c': 1,
         '.': 1})

So tokenization is happening at charecter level. so we need an interator of iterator of tokens. Lets tokenize again.

In [36]:
field = torchtext.data.Field()
tokens = field.tokenize(txt)

# we use a list of list of tokens
field.build_vocab([tokens])

#check the word frequencies
print(f'frequencies: {field.vocab.freqs}')

# words in vocab
print(f'vocab words: {list(field.vocab.stoi.keys())}')

frequencies: Counter({'This': 1, 'is': 1, 'simple': 1, 'sentence': 1, '.': 1})
vocab words: ['<unk>', '<pad>', '.', 'This', 'is', 'sentence', 'simple']


Let us tokenize and numericalize

In [37]:
data = field.numericalize([tokens])
print('data.shape: ', data.shape)
data

data.shape:  torch.Size([5, 1])


tensor([[3],
        [4],
        [6],
        [5],
        [2]])